In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

# Install if in Google colab notebook
if 'google.colab' in sys.modules:
    os.system('python -m pip install --upgrade git+https://github.com/manzt/hg')

## Synced heatmaps

In [ ]:
import hg
import hg.tilesets

remote_tileset = hg.tilesets.remote(
    uid='CQMd6V_cRw6iCI_-Unl3PQ',
    server='https://higlass.io/api/v1/',
)

view1 = hg.view(
    hg.Tracks(
        top=[hg.track('top-axis')],
        center=[remote_tileset.track('heatmap')],
    ),
    width=6,
)

view2 = view1.move(x=6)

view_lock = hg.lock(view1, view2) # lock a variable number of views

# lock both zoom and location
hg.viewconf(view1, view2).locks(view_lock)

In [ ]:
# lock zoom only
hg.viewconf(view1, view2).locks(zoom=view_lock)

In [ ]:
# lock location only
hg.viewconf(view1, view2).locks(location=view_lock)

In [ ]:
view3 = view1.move(y=1)
view4 = view2.move(y=1)
hg.viewconf(view1, view2, view3, view4).locks(
    hg.lock(view1, view2),
    hg.lock(view3, view4),
)

## Value scale syncing

In [ ]:
# Create Tracks

# creates a hg.Track without a tileset
t1 = hg.track('top-axis')

# Creates a hg.RemoteTileset object
remote_tileset = hg.tilesets.remote(
    uid='CQMd6V_cRw6iCI_-Unl3PQ',
    server="https://higlass.io/api/v1/", # optional, "http://higlass.io/api/v1/" default
)

# Tileset.track() creates a hg.Track object binding the parent Tileset 
t2 = remote_tileset.track('heatmap', width=250)

# set the options for the returned track
t2 = t2.opts(valueScaleMax=0.5) 


# Create Views 

# Positional arguments for `hg.view` are overloaded. Keyword arguments are 
# forwarded to the creation of the `hg.View`. Layout fields (`x`, `y`, 
# `width`, `height`) may also be assigned.

# (1) Track objects (positioning guessed based on track type)
view1 = hg.view(t1, t2, width=6)

# (2) (Track, position) tuple
view2 = hg.view((t1, 'top'), t2, width=6, x=6)

# (3) hg.Tracks object
view3 = hg.view(hg.Tracks(top=[t1], center=[t2]), width=6, x=6)

scale_value_lock = hg.lock((view1, t2), (view2, t2)) # (View, Track) tuples -> ScaleValueLock

hg.viewconf(view1, view2).locks(scale_value_lock)

## Remote heatmaps

In [ ]:
tset1 = hg.tilesets.remote(uid='CQMd6V_cRw6iCI_-Unl3PQ', server='https://higlass.io/api/v1/')
tset2 = hg.tilesets.remote(uid='QvdMEvccQuOxKTEjrVL3wA')

t1 = tset1.track('heatmap', height=210)
t2 = tset2.track('heatmap', height=210)
t3 = hg.divide(t1, t2).opts(
    colorRange=['blue', 'white'],
    valueScaleMin=0.1,
    valueScaleMax=10,
)

domain = (7e7, 8e7)

v1 = hg.view(t1, x=0, width=4).domain(x=domain)
v2 = hg.view(t3, x=4, width=4).domain(x=domain)
v3 = hg.view(t2, x=8, width=4).domain(x=domain)

hg.viewconf(v1, v3, v2).locks(hg.lock(v1, v2, v3))

## Extract track from another view config

In [ ]:
import hg
import urllib.request as urllib

url = 'https://gist.githubusercontent.com/manzt/c2c498dac3ca9804a2b8e4bc1af3b55b/raw/ee8426c9728e875b6f4d65030c61181c6ba29b53/example.json'
with urllib.urlopen(urllib.Request(url)) as response:
    raw = response.read()
    config = hg.Viewconf.parse_raw(raw)
        
config.views[0].tracks.top[0] # a pydantic BaseModel https://pydantic-docs.helpmanual.io/usage/models/

In [ ]:
gene_annotation_track = config.views[0].tracks.top[0].properties(height=200)
hg.viewconf(hg.view(gene_annotation_track))

## Remote bigWig tiles ---- TODO , old higlass-python api

In [ ]:
import higlass
from higlass.client import View, Track
import higlass.tilesets

ts1 = higlass.tilesets.bigwig(
    'http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878InputStdSig.bigWig')
view1 = View([Track('horizontal-bar', tileset=ts1)])
display, server, viewconf = higlass.display([view1], fuse=False)

display

## Local cooler files

This section describes how to load cooler files that are on the same filesystem as the Jupyter notebook. If the Jupyter kernel is not running on localhost, see the section on [Remote Jupyter notebooks](https://docs-python.higlass.io/getting_started.html#remote-jupyter-notebook) in the docs.

In [ ]:
ts1 = higlass.tilesets.cooler('../test/data/Dixon2012-J1-NcoI-R1-filtered.100kb.multires.cool')
tr1 = Track('heatmap', tileset=ts1)
view1 = View([tr1])
display, server, viewconf = higlass.display([view1])

display

## Local bigWig files (with chromsizes)

In [ ]:
chromsizes = [
 ('chr1', 249250621),
 ('chr2', 243199373),
 ('chr3', 198022430),
 ('chr4', 191154276),
 ('chr5', 180915260),
 ('chr6', 171115067),
 ('chr7', 159138663),
 ('chr8', 146364022),
 ('chr9', 141213431),
 ('chr10', 135534747),
 ('chr11', 135006516),
 ('chr12', 133851895),
 ('chr13', 115169878),
 ('chr14', 107349540),
 ('chr15', 102531392),
 ('chr16', 90354753),
 ('chr17', 81195210),
 ('chr18', 78077248),
 ('chr20', 63025520),
 ('chr19', 59128983),
 ('chr21', 48129895),
 ('chr22', 51304566),
 ('chrX', 155270560),
 ('chrY', 59373566),
 ('chrM', 16571),
]

bigwig_fp = '../test/data/wgEncodeCaltechRnaSeqHuvecR1x75dTh1014IlnaPlusSignalRep2.bigWig'
ts = higlass.tilesets.bigwig(bigwig_fp, chromsizes=chromsizes)
cs = higlass.tilesets.ChromSizes(chromsizes)

view1 = View([
    Track('top-axis'),
    Track('horizontal-bar', tileset=ts),
    Track('horizontal-chromosome-labels', position='top', tileset=cs)
])
display, server, viewconf = higlass.display([view1])

display

## Local bedlike data

In [ ]:
from higlass.client import View, Track
from higlass.inline_tiles import bedtiles
import higlass

bed = [['chr1', 1000, 2000, 'item #1', '.', '+'],
       ['chr2', 3000, 3500, 'item #1', '.', '-']]

chroms = [['chr1', 2100], ['chr2', 4000]]


data = bedtiles(bed, chroms)
track = Track(track_type='bedlike', position='top',
              height=50, data=data, options={"minusStrandColor": "red"})


d,s,v = higlass.display([[track]])
d

## Custom data

In [ ]:
import numpy as np

dim = 2000
I, J = np.indices((dim, dim))
data = (
    -(J + 47) * np.sin(np.sqrt(np.abs(I / 2 + (J + 47)))) 
    - I * np.sin(np.sqrt(np.abs(I - (J + 47))))
)

In [ ]:
import clodius.tiles.npmatrix
from higlass.tilesets import Tileset

ts = Tileset(
    tileset_info=lambda: clodius.tiles.npmatrix.tileset_info(data),
    tiles=lambda tids: clodius.tiles.npmatrix.tiles_wrapper(data, tids)
)

display, server, viewconf = higlass.display([
    View([
        Track(track_type='top-axis', position='top'),
        Track(track_type='left-axis', position='left'),
        Track(track_type='heatmap', 
                  position='center',
                  tileset=ts,
                  height=250,
                 options={ 'valueScaleMax': 0.5 }),

    ])
])
display

In [ ]:
%%time

import requests
import json

url = 'http://localhost:{}/api/v1/register_url/'.format(server.port)
print('url:', url)
ret = requests.post(url,
             json={
                 'fileUrl': 'http://hgdownload.cse.ucsc.edu/goldenpath/hg19/encodeDCC/wgEncodeSydhTfbs/wgEncodeSydhTfbsGm12878InputStdSig.bigWig',
                 #'fileUrl': 'http://localhost:8111/wgEncodeCaltechRnaSeqHuvecR1x75dTh1014IlnaPlusSignalRep2.bigWig',
                 'filetype': "bigwig"
             })

print("content:", ret.content)

uid = json.loads(ret.content)['uid']
url = 'http://localhost:{}/api/v1/tiles/?d={}.2.0&d={}.2.1&d={}.2.2'.format(server.port, uid, uid, uid)
#url = "http://localhost:8111/wgEncodeCaltechRnaSeqHuvecR1x75dTh1014IlnaPlusSignalRep2.bigWig"
req = requests.get(url)
print('req', req.content)

"""
url = 'http://localhost:{}/api/v1/tileset_info/?d={}'.format(server.port, uid)
print('url:', url);
req = requests.get(url)
print('req', req.content)
"""